In [ ]:
# import packages:

import numpy as np
import pandas as pd
import math
import os
import glob

In [ ]:
# function to return the stem energy based on nearest-neighbor interactions:

def stem_energy(sp):
    se = 0
    if len(sp) > 1:
        for i in range(1, len(sp)):
            if sp[i] == "AU":
                if sp[i-1] == "AU": 
                    se += 0.9
                if sp[i-1] == "CG":
                    se += 2.2
                if sp[i-1] == "GC":
                    se += 2.1
                if sp[i-1] == "UA":
                    se += 1.1
                if sp[i-1] == "GU":
                    se += 0.6
                if sp[i-1] == "UG":
                    se += 1.4
            if sp[i] == "CG":
                if sp[i-1] == "AU": 
                    se += 2.1
                if sp[i-1] == "CG":
                    se += 3.3
                if sp[i-1] == "GC":
                    se += 2.4
                if sp[i-1] == "UA":
                    se += 2.1
                if sp[i-1] == "GU":
                    se += 1.4
                if sp[i-1] == "UG":
                    se += 2.1
            if sp[i] == "GC":
                if sp[i-1] == "AU": 
                    se += 2.4
                if sp[i-1] == "CG":
                    se += 3.4
                if sp[i-1] == "GC":
                    se += 3.3
                if sp[i-1] == "UA":
                    se += 2.2
                if sp[i-1] == "GU":
                    se += 1.5
                if sp[i-1] == "UG":
                    se += 2.5
            if sp[i] == "UA":
                if sp[i-1] == "AU": 
                    se += 1.3
                if sp[i-1] == "CG":
                    se += 2.4
                if sp[i-1] == "GC":
                    se += 2.1
                if sp[i-1] == "UA":
                    se += 0.9
                if sp[i-1] == "GU":
                    se += 1.0
                if sp[i-1] == "UG":
                    se += 1.3
            if sp[i] == "GU":
                if sp[i-1] == "AU": 
                    se += 1.3
                if sp[i-1] == "CG":
                    se += 2.5
                if sp[i-1] == "GC":
                    se += 2.1
                if sp[i-1] == "UA":
                    se += 1.4
                if sp[i-1] == "GU":
                    se += 0.5
                if sp[i-1] == "UG":
                    se += -1.3
            if sp[i] == "UG":
                if sp[i-1] == "AU": 
                    se += 1.0
                if sp[i-1] == "CG":
                    se += 1.5
                if sp[i-1] == "GC":
                    se += 1.4
                if sp[i-1] == "UA":
                    se += 0.6
                if sp[i-1] == "GU":
                    se += -0.3
                if sp[i-1] == "UG":
                    se += 0.5
    return se

In [ ]:
# function to read in .ct file and give a list of known structure stems:

def actual_stems(seq_ss, seq_ps):    # seq_ss: secondary structure, seq_ps: primary structure (sequence)
    
    with open(subdirectory+"/"+seq_ss) as file:
        ss_lines = file.readlines()
    
    with open(subdirectory+"/"+seq_ps) as file:
        ps_lines = file.readlines()
    
    rna = ps_lines[1]
    
    stems_actual = []

    sip = False                       # stem in progress?
    sl = 0                            # stem length
    sp = []                           # stem pairs
    last_line = [0, 0, 0, 0, 0, 0]    # initiate last line

    for i in range(0, len(ss_lines)):
        line = ss_lines[i].strip().split()
        
        if (int(line[4]) != 0 and sip == False):
            sip = True
            temp = [int(line[0]), int(line[4])]
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')):
                sp.append("GC")
            if (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("CG")
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("GU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("UG")
            if (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("AU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sp.append("UA")
            sl += 1
            
        if (int(line[4]) != 0 and sip == True and (int(last_line[4])-int(line[4]) == 1)):
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')):
                sp.append("GC")
            if (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("CG")
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("GU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("UG")
            if (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("AU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sp.append("UA")
            sl += 1
            
        if (int(line[4]) == 0 and sip == True):
            sip = False
            temp.append(sl)
            temp.append(int(temp[1]-temp[0]-2*sl))
            temp.append(stem_energy(sp))
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            sl = 0
            sp = []
            
        if ((int(last_line[4])-int(line[4]) != 1) and int(last_line[4]) != 0  and sip == True):
            temp.append(sl)
            temp.append(int(temp[1]-temp[0]-2*sl))
            temp.append(stem_energy(sp))
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            temp = [int(line[0]), int(line[4])]
            sl = 0
            sp = []
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')):
                sp.append("GC")
            if (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("CG")
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("GU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')):
                sp.append("UG")
            if (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')):
                sp.append("AU")
            if (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sp.append("UA")
            sl += 1
        
        last_line = line
        
    return stems_actual

In [ ]:
# function to read in .fasta file and generate list of potential stems at least 3 base-pairs long:

def potential_stems(seq_ps):
    
    with open(subdirectory+"/"+seq_ps) as file:
        lines = file.readlines()
    
    rna = lines[1]
    
    matrix = np.zeros((len(rna),len(rna)))
    for diag in range(0, len(matrix)):
        for row in range(0, len(matrix)-diag):
            col = row + diag
            base1 = rna[row]
            base2 = rna[col]
            if row != col:
                if ((base1 == ("A" or "a")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("A" or "a"))) or ((base1 == ("G" or "g")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("G" or "g"))) or ((base1 == ("G" or "g")) and (base2 == ("C" or "c"))) or ((base1 == ("C" or "c")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 1
    
    stems_potential = []
    mu = 0

    for row in range(0, len(matrix)):
        for col in range (row, len(matrix)):
            if row != col:
                if matrix[row][col] != 0:
                    sp = []                 # stem pairs
                    temp_row = row
                    temp_col = col
                    stem = [row+1,col+1,0,0,0]
                    length = 0
                    while (matrix[temp_row][temp_col] != 0) and (temp_row != temp_col):
                        base1 = rna[temp_row]
                        base2 = rna[temp_col]
                        if (base1 == ('G' or 'g') and base2 == ('C' or 'c')):
                            sp.append("GC")
                        if (base1 == ('C' or 'c') and base2 == ('G' or 'g')):
                            sp.append("CG")
                        if (base1 == ('G' or 'g') and base2 == ('U' or 'u')):
                            sp.append("GU")
                        if (base1 == ('U' or 'u') and base2 == ('G' or 'g')):
                            sp.append("UG")
                        if (base1 == ('A' or 'a') and base2 == ('U' or 'u')):
                            sp.append("AU")
                        if (base1 == ('U' or 'u') and base2 == ('A' or 'a')):
                            sp.append("UA")
                        length += 1
                        temp_row += 1
                        temp_col -= 1
                        if length >= 3:
                            stem[2] = int(length)
                            stem[3] = int(col-row-2*length)
                            stem[4] = stem_energy(sp)
                            stems_potential.append(stem.copy())
                    if stem_energy(sp) > mu:
                        mu = stem_energy(sp)
    
    return [stems_potential, mu, rna, len(rna)]

In [ ]:
# function to generate energy per in-line pseudoknotted helix of length n:

def pseudoknot_sub_penalty(length):
    return np.exp(0.572992*length+0.219677)

In [ ]:
# function to generate list of potential stem pairs that form pseudoknots:

def potential_pseudoknots(stems_potential, gamma):

    pseudoknots_potential = []

    for i in range(len(stems_potential)):
        for j in range(i + 1, len(stems_potential)):
            
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            i_a = stem1[0]
            j_a = stem1[1]
            i_b = stem2[0]
            j_b = stem2[1]
    
            pseudoknot = [i,j,1]
            if (i_a < i_b and i_b < j_a and j_a < j_b) or (i_b < i_a and i_a < j_b and j_b < j_a):
                
                pseudoknot[2] = gamma*np.log(stem1[2]*pseudoknot_sub_penalty(stem1[2])**2+stem2[2]*pseudoknot_sub_penalty(stem2[2])**2)
    
            pseudoknots_potential.append(pseudoknot)
            
    return pseudoknots_potential

In [ ]:
# function to generate list of stem pairs that overlap:

def potential_overlaps(stems_potential):
    
    overlaps_potential = []
    overlap_penalty = 1e6

    for i in range(len(stems_potential)):
        for j in range(i+1, len(stems_potential)):
    
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            overlap = [i, j, 0]
    
            stem1_cspan1 = set(range(stem1[1]-int(stem1[2])+1, stem1[1]+1))
            stem2_cspan1 = set(range(stem2[1]-int(stem2[2])+1, stem2[1]+1))
            
            stem1_cspan2 = set(range(stem1[0], stem1[0]+int(stem1[2])))
            stem2_cspan2 = set(range(stem2[0], stem2[0]+int(stem2[2])))
    
            if (len(stem1_cspan1 & stem2_cspan1) != 0) or (len(stem1_cspan2 & stem2_cspan2) != 0)  or (len(stem1_cspan1 & stem2_cspan2) != 0) or (len(stem1_cspan2 & stem2_cspan1) != 0):
        
                overlap[2] = overlap_penalty
        
            overlaps_potential.append(overlap)
            
    return overlaps_potential

In [ ]:
def loop_penalty(ll):
    lp = 0
    if ll == 0 or ll == 1 or ll == 2:
        lp = 1000
    if ll == 3:
        lp = 7.4
    if ll == 4:
        lp = 5.9
    if ll == 5:
        lp = 4.4
    if ll == 6:
        lp = 4.3
    if ll >= 7:
        lp = 4.1
    return lp

In [ ]:
# function to generate the Hamiltonian of a given RNA structure from potential stems, overlaps, and pseudoknots:

def model(stems_potential, pseudoknots_potential, overlaps_potential, mu):
    
    L = {}
    Q = {}
    alpha = 1
    beta = 1
    k = 0

    for i in range(0, len(stems_potential)):
        k_i = stems_potential[i][4]
        lp  = stems_potential[i][3]
        L[str(i)] = alpha*((k_i-mu)**2)-beta*(k_i-loop_penalty(stems_potential[i][2]))
        for j in range(i+1, len(stems_potential)):
            Q[(str(i), str(j))] = pseudoknots_potential[k][2]+overlaps_potential[k][2]
            k += 1
    
    return L, Q

In [ ]:
# function to evaluate the energy of the known structure under the model Hamiltonian:

def energy(stems_actual, gamma):
    
    alpha = 1
    beta = 1
    k = 0
    pseudoknots_actual = potential_pseudoknots(stems_actual, gamma)
    cost = 0
    mu = max(list(map(list, zip(*stems_actual)))[2])
        
    for i in range(0, len(stems_actual)):
        k_i = stems_actual[i][4]
        cost += alpha*((k_i-mu)**2)-beta*(k_i-loop_penalty(stems_actual[i][2]))
        for j in range(i+1, len(stems_actual)):
            cost += pseudoknots_actual[k][2]
            k += 1
    
    return cost

In [ ]:
# function to compare actual and predicted structure based on comparison of base-pairs:

def evaluation_1(stems_actual, stems_potential):
    
    bp_actual = []
    bp_predicted = []

    for i in range(0, len(stems_actual)):
        for j in range(0, stems_actual[i][2]):
            bp_actual.append((stems_actual[i][0]+j, stems_actual[i][1]-j))
        
    for i in range(0, len(stems_potential)):
        for j in range(0, stems_potential[i][2]):
            bp_predicted.append((stems_potential[i][0]+j, stems_potential[i][1]-j))
            
    C = 0    # number of correctly identified base pairs
    M = 0    # number of the predicted base pairs missing from the known structure
    I = 0    # number of non-predicted base pairs present in the known structure

    for i in range(0, len(bp_predicted)):
        if bp_predicted[i] in bp_actual:
            C += 1
        else:
            M += 1

    for i in range(0, len(bp_actual)):
        if bp_actual[i] not in bp_predicted:
            I += 1
            
    ppv = C/(C+M)
    sensitivity = C/(C+I)
    
    return [ppv, sensitivity]

In [ ]:
# function to compare actual and predicted structure based on comparison of bases involved in pairing:

def evaluation_2(stems_actual, stems_predicted):
    
    pb_actual = []
    pb_predicted = []

    for i in range(0, len(stems_actual)):
        for j in range(0, stems_actual[i][2]):
            pb_actual.append(stems_actual[i][0]+j)
            pb_actual.append(stems_actual[i][1]-j)
        
    for i in range(0, len(stems_predicted)):
        for j in range(0, stems_predicted[i][2]):
            pb_predicted.append(stems_predicted[i][0]+j)
            pb_predicted.append(stems_predicted[i][1]-j)
            
    C = 0    # number of correctly identified bases that are paired
    M = 0    # number of the predicted paired bases missing from the known structure
    I = 0    # number of non-predicted paired bases present in the known structure

    for i in range(0, len(pb_predicted)):
        if pb_predicted[i] in pb_actual:
            C += 1
        else:
            M += 1

    for i in range(0, len(pb_actual)):
        if pb_actual[i] not in pb_predicted:
            I += 1
            
    ppv = C/(C+M)
    sensitivity = C/(C+I)
    
    return [ppv, sensitivity]

In [ ]:
def connectivity_table(bpRNA_id, sequence, stems, t):
    print(len(sequence), bpRNA_id, file=open(subdirectory+"/"+bpRNA_id+"_"+t+".ct", "w"))
    for i in range(0, len(sequence)):
        pair = 0
        for j in stems:
            for k in range(j[0], j[0]+j[2]):
                if i+1 == k:
                    pair = j[1]+j[0]-k
            for k in range(j[1]-j[2]+1, j[1]+1):
                if i+1 == k:
                    pair = j[0]+j[1]-k
        print(i+1, sequence[i], i, i+2, pair, i+1, file=open("./results/cts/"+bpRNA_id+"_"+t+".ct", "a"))

In [ ]:
# connecting with D-Wave:

from dwave.cloud import Client

client = Client.from_config(token="DEV-6b38e4697eaa586b361595c629788f595b810a14")
client.get_solvers()

from dwave.system.samplers import DWaveSampler
from dwave.system.samplers import LeapHybridSampler
from dwave.system.composites import EmbeddingComposite

import dimod

sampler_q = EmbeddingComposite(DWaveSampler(token="DEV-6b38e4697eaa586b361595c629788f595b810a14", solver={'topology__type': 'pegasus'}))
sampler_h = LeapHybridSampler(token="DEV-6b38e4697eaa586b361595c629788f595b810a14")

In [ ]:
# loop over all structures, submitting to D-Wave, recording predicted stems and energies, recording comparisons between known and precicted structures:

pk = ["wPKs", "woutPKs"]
s  = ["s", "m", "l"]
pkps  = [-1.0, -0.5, 0.0, 0.5, 1.0]
pl = ["n1", "np5", "0", "pp5", "p1"]

data = []

for a in range(0, len(pk)):
    for b in range(0, len(s)):
        for c in range(0, len(pl)):
            subdirectory = './structures/'+pk[a]+'/'+s[b]

            ct = [f for f in os.listdir(subdirectory) if f.endswith('.ct.txt')]
            fasta = [f for f in os.listdir(subdirectory) if f.endswith('.fasta.txt')]

            bprna_id = []
            size = []
            pks = []
            pk_penalty = []

            penalty = pl[c]

            for i in range(0, len(ct)):
                bprna_id.append(ct[i].split('.')[0])
                size.append(subdirectory.split("/")[3])
                if subdirectory.split("/")[2] == "wPKs":
                    pks.append("T")
                else:
                    pks.append("F")
                pk_penalty.append(penalty)
            
            stems_a    = []
            energies_a = []
            stems_p    = []
            pks_p      = []
            ols_p      = []
            models     = []
            for index in range(0, len(ct)):
                stems_a.append(actual_stems(ct[index], fasta[index]))
                energies_a.append(energy(stems_a[index], pkps[c]))
                stems_p.append(potential_stems(fasta[index]))
                pks_p.append(potential_pseudoknots(stems_p[index][0], pkps[c]))
                ols_p.append(potential_overlaps(stems_p[index][0]))
                models.append(model(stems_p[index][0], pks_p[index], ols_p[index], stems_p[index][1]))
                                
            problem = []

            for i in range(0, len(ct)):
                problem.append(dimod.BinaryQuadraticModel(models[i][0], models[i][1], vartype = 'BINARY', offset = 0.0))    
            
            stems_f = []
            min_time = []

            for i in range(0, len(ct)):
                try:
                    #sampleset = sampler_q.sample(problem[i], num_reads=1000)
                    sampleset = sampler_h.sample(problem[i]) # hybrid
                    #min_time.append("placeholder")
                    min_time.append(sampler_h.min_time_limit(problem[i])) # hybrid

                    rn = 0
                    for datum in sampleset.data(['sample', 'energy', 'num_occurrences']):
                        if rn == 0:
                            results = datum.sample
                            predicted_energy = datum.energy
                        rn += 1
    
                    f_stems = []

                    for j in range(0, len(results)):
                        if results[str(j)] == 1:
                            f_stems.append(stems_p[i][0][j])
            
                    stems_f.append([f_stems, predicted_energy])
                
                    connectivity_table(bprna_id[i], stems_p[i][2], stems_f[i][0], "predicted")
                    connectivity_table(bprna_id[i], stems_p[i][2], stems_a[i], "actual")
                except:
                    print("no embedding found, skipping...")
            
            metrics_1 = []
            metrics_2 = []

            for index in range(0, len(ct)):
                try:
                    metrics_1.append(evaluation_1(stems_a[index], stems_f[index][0]))
                    metrics_2.append(evaluation_2(stems_a[index], stems_f[index][0]))
                except:
                    print("no structure found, skipping...")
            
            headers = ["bprna_id", "sequence", "length", "size", "pks", "pk_penalty", "stems_actual", "energy_actual", "stems_predicted", "energy_predicted", "time_to_solution", "ppv_bp", "sensitivity_bp", "ppv_pb", "sensitivity_pb"]
            data.append(pd.DataFrame(list(zip(bprna_id, list(map(list, zip(*stems_p)))[2], list(map(list, zip(*stems_p)))[3], size, pks, pk_penalty, stems_a, energies_a, list(map(list, zip(*stems_f)))[0], list(map(list, zip(*stems_f)))[1], min_time, list(map(list, zip(*metrics_1)))[0], list(map(list, zip(*metrics_1)))[1], list(map(list, zip(*metrics_2)))[0], list(map(list, zip(*metrics_2)))[1])), columns = headers))

In [ ]:
data_ = pd.concat(data)
data_.to_csv('./results/results.csv', index=False)